<a href="https://colab.research.google.com/github/ds4geo/ds4geo/blob/master/WS%202020%20Course%20Notes/Session%205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To Do

* Student assignment submissions link.

**Other**
* Submission method
* References


* Create separate solutions notebook
* Enable output cells
* Final go-through and checks

# **Data Science for Geoscientists - Winter Semester 2020**
# **Session 5 - Data Story Telling - 4th November 2020**

Weather assignment discuss

Simple stats, mean --> kde

Data story telling

How do I do that open workshop

Assignment: weather 2

# Part 5.1 - Weather data assignment review - *Discussion*
We will look at the assignment submissions and discuss the conclusions (what is the weather like in these cities?) and the effectiveness of the analysis, visualisations and story-telling.

# Part 5.2 - Statistics: Describing data - *Workshop*
As we see in the 

In [10]:
import pandas as pd
from scipy import stats

In [5]:
# Load the data from here: "/content/sample_data/anscombe.json"
# format is json - find a pandas function to load it
dat = pd.read_json("/content/sample_data/anscombe.json")

In [12]:
# Select one of the datasets:
series_names = ["I", "II", "III", "IV"]
for n in series_names:
  #print(dat.loc[dat.Series==n].std())
  sdat = dat.loc[dat.Series==n]
  print(stats.linregress(sdat.X, sdat.Y))

LinregressResult(slope=0.5002727272727273, intercept=2.9975454545454543, rvalue=0.81618645422891, pvalue=0.0021813202598156855, stderr=0.11804977471937325)
LinregressResult(slope=0.5000000000000001, intercept=3.000909090909089, rvalue=0.816236506000243, pvalue=0.0021788162369107845, stderr=0.11796374596764074)
LinregressResult(slope=0.4997272727272729, intercept=3.002454545454544, rvalue=0.8162867394895984, pvalue=0.002176305279228015, stderr=0.11787766222100221)
LinregressResult(slope=0.4999090909090909, intercept=3.0017272727272726, rvalue=0.8165214368885028, pvalue=0.002164602347197222, stderr=0.11781894172968553)
